# Trader Bot with Binance API

In this work, crypto data will be imported using Binance API and and a trader bot will be created for automated with python.

**See also** Technical Indicators for Financial Analysis with Python (Jupyter): (the link will be uploaded.)

**See also** Gate.io Trader Bot with Python: (the link will be uploaded.)

**See also** Data Visualization for Financial Analysis with Python (Jupyter): (the link will be uploaded.)

<br><br>

**WARNING!!**

<li>The strategies used in this section are <b>not investment advices</b>. They are shown for only educational purposes.
<li>The bot makes real transactions. Therefore, be sure to understand them clearly before implementation. Otherwise, you can lose your money.

<br><br>

## 1. Importing Libraries and Data

<br>**Importing Libraries**

In [65]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import sqlalchemy
import time
from datetime import datetime
import os
import asyncio
import warnings
warnings.filterwarnings('ignore')
from mplfinance.original_flavor import candlestick_ohlc
import pandas_ta as pta

In [2]:
#pip install python-binance

In [3]:
from binance import Client
from binance import BinanceSocketManager

**Note** that some libraries should be installed with pip install.

**Warning:** You should pip install <b>python-binance</b> to be able to import binance

<br>**Creating Connection**

In [ ]:
api_key = "ENTER YOUR api_key"
api_secret = "ENTER YOUR api_secret"

In [5]:
client = Client(api_key, api_secret)


<br><br>**Optional: Importing data with BinanceSocketManager**

There are two ways to import data from Binance. Either using Socket or getting historical data.<br>
<li> Using Socket Manager you can store your data to a local database.
<li> Getting historical data can provide historical and present data. If you do not want to store data in a local database then it is more feasible to choose this option.

In [21]:
bsm = BinanceSocketManager(client)
socket = bsm.trade_socket("MINAUSDT")
engine = sqlalchemy.create_engine("sqlite:///BTCUSDTstream.db")
await socket.__aenter__()
msg = await socket.recv()
msg

{'e': 'trade',
 'E': 1657721923714,
 's': 'MINAUSDT',
 't': 21862622,
 'p': '0.61600000',
 'q': '229.80000000',
 'b': 140915181,
 'a': 140915244,
 'T': 1657721923713,
 'm': True,
 'M': True}

In [ ]:
pd.read_sql("MINAUSDT", engine)

<br><li> As seen, the data is provided in json format. We can create a pandas dataframe with a function

In [6]:
def create_df(msg):
    # creating pandas dataframe
    df = pd.DataFrame([msg])
    
    # filtering the features we want
    df = df.loc[:,["s","E","p"]]
    
    # renaming column names
    df.columns = ["Symbol", "Time","Price"]
    
    # replacing data types
    df.Price = df["Price"].astype(float)
    df.Time = pd.to_datetime(df.Time,unit = "ms")
    
    return df

In [26]:
create_df(msg)

,Symbol,Time,Price
0,MINAUSDT,2022-07-13 14:18:43.714,0.616


In [30]:
create_df(msg).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Symbol  1 non-null      object        
 1   Time    1 non-null      datetime64[ns]
 2   Price   1 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 152.0+ bytes


<br><br>**Importing Historical Data (inc. the most recent)**

We can import historical data with a single code.

In [28]:
client.get_historical_klines("ETHUSDT", "15m", "60 min ago UTC+")

[[1657719900000,
  '1037.00000000',
  '1046.16000000',
  '1034.29000000',
  '1041.73000000',
  '63858.18320000',
  1657720799999,
  '66451551.53421900',
  42034,
  '30673.69080000',
  '31921165.39044400',
  '0'],
 [1657720800000,
  '1041.73000000',
  '1048.97000000',
  '1037.74000000',
  '1041.96000000',
  '40773.17680000',
  1657721699999,
  '42472233.31233400',
  29525,
  '21064.39330000',
  '21938229.65791000',
  '0'],
 [1657721700000,
  '1041.96000000',
  '1043.00000000',
  '1032.80000000',
  '1036.92000000',
  '28000.53860000',
  1657722599999,
  '29036452.48743600',
  22632,
  '13430.70630000',
  '13927484.21765300',
  '0'],
 [1657722600000,
  '1036.91000000',
  '1041.25000000',
  '1036.01000000',
  '1041.06000000',
  '4787.25610000',
  1657723499999,
  '4968746.59110200',
  3015,
  '2422.22210000',
  '2513994.31848400',
  '0']]

<br><br> This function requires the pair name we want (ex: "ETHUSDT"), the time interval (ex: "15m") and how further we want to go back in time (ex:"60 minutes"). 

As you can see, this function returns a list. To understand their meanings you can check the documentation, but we can create a dataframe for better understanding. To do that we can define a function.

In [7]:
 def klines_table(currency,interval,timeago):
    # creating pandas dataframe
    frame = pd.DataFrame(client.get_historical_klines(str(currency), str(interval), str(timeago)+" min ago UTC+"))    
    
    # changing column names
    frame.columns = ["Time","Open", "High", "Low", "Close","Volume", "Close_time", "volume", "Number_of_trades", "Taker_buy_base_asset_vol", "Taker_buy_quote_asset_vol", "Ignore"]
    
    # setting time as index and setting as time
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit ="ms")
    
    # changing all values to float
    frame = frame.astype(float)
    return frame

In [32]:
klines_table("MINAUSDT", "15m",60)

,Open,High,Low,Close,Volume,Close_time,volume,Number_of_trades,Taker_buy_base_asset_vol,Taker_buy_quote_asset_vol,Ignore
Time,,,,,,,,,,,
2022-07-13 13:45:00,0.610,0.619,0.608,0.618,159188.5,1.657721e+12,97781.7611,566.0,89559.7,55044.8583,0.0
2022-07-13 14:00:00,0.617,0.621,0.615,0.618,70783.7,1.657722e+12,43747.6022,305.0,36730.6,22717.4072,0.0
2022-07-13 14:15:00,0.618,0.619,0.615,0.618,35093.4,1.657723e+12,21655.1132,172.0,18595.8,11480.4054,0.0
2022-07-13 14:30:00,0.618,0.625,0.616,0.624,45722.3,1.657723e+12,28428.4059,212.0,31952.4,19865.6398,0.0


In [33]:
klines_table("MINAUSDT", "15m",60).info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4 entries, 2022-07-13 13:45:00 to 2022-07-13 14:30:00
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Open                       4 non-null      float64
 1   High                       4 non-null      float64
 2   Low                        4 non-null      float64
 3   Close                      4 non-null      float64
 4   Volume                     4 non-null      float64
 5   Close_time                 4 non-null      float64
 6   volume                     4 non-null      float64
 7   Number_of_trades           4 non-null      float64
 8   Taker_buy_base_asset_vol   4 non-null      float64
 9   Taker_buy_quote_asset_vol  4 non-null      float64
 10  Ignore                     4 non-null      float64
dtypes: float64(11)
memory usage: 384.0 bytes


<br> Now the table is more understandable. We could also set "Close_time" as time, but we will focus on price parameters.

**Note that** at the last row "Close" represents the current price, so each time we rerun this function, the "Close" price will be updated because the current time interval has not been closed yet.

<br><br>

**Checking Assets in the Wallet**

We can check our assets with a single function. Note that it may give a huge amount of data, because it shows also the pairs we have not invested.

In [36]:
client.get_account()["balances"][0:3]

[{'asset': 'BTC', 'free': '0.00000768', 'locked': '0.00000000'},
 {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'ETH', 'free': '0.00006328', 'locked': '0.00000000'}]

<br> We can define a function to make it clear.

In [8]:
def myasset():
    # importing the assets
    df = pd.DataFrame(client.get_account()["balances"])
    
    # setting index as asset pair
    df = df.set_index("asset")
    
    # setting data types as float
    df = df.astype(float)
    
    # adding a column to see our total asset
    df["total"] = df["free"] + df["locked"]
    
    # sorting the assets by our remaining amount
    df = df.sort_values("total",ascending = False)
    
    # filtering the assets that we do not possess
    df = df[df["total"] >0]
    
    # returning only four digits of the values
    return df.round(4)

In [39]:
myasset().tail(5)

,free,locked,total
asset,,,
ETH,0.0001,0.0,0.0001
LUNA,0.0001,0.0,0.0001
SUPER,0.0000,0.0,0.0000
BTC,0.0000,0.0,0.0000
DASH,0.0000,0.0,0.0000


<br><br>

**Checking Trades**

You can import your trading transactions with a single function. You can also create a function to clean your dataframe if required.

In [10]:
pd.DataFrame(client.get_my_trades(symbol="MINAUSDT")).tail(3)

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch
497,MINAUSDT,7630719,40896597,-1,4.14500000,65.30000000,270.66850000,0.00042362,BNB,1634935184420,False,False,True
498,MINAUSDT,7630720,40896597,-1,4.14500000,25.00000000,103.62500000,0.00016218,BNB,1634935184420,False,False,True
499,MINAUSDT,7630740,40896597,-1,4.14500000,36.10000000,149.63450000,0.00023418,BNB,1634935184426,False,True,True


<br><br>

# 2. TRADING BOT

A trading bot can trade automatically when the conditions defined by us satisfied. These conditions can be a fixed price or a dynamic price calculated by technical indicators.

**WARNING!!**

<li>The strategies used in this section are <b>not investment advices</b>. They are shown for only educational purposes.
<li>The bot makes real transactions. Therefore, be sure to understand them clearly before implementation. Otherwise, you can lose your money.

<br><br><br>

**Buy/Sell Function**

We can begin with a simple bot that buys/sells at a certain price. The following codes can be used to make a real transaction.

**order = client.create_order(symbol = "BNBUSDT",
                                       side = "BUY",
                                       type = "MARKET",
                                       quantity = "0.05")**

**order = client.create_order(symbol = "BNBUSDT",
                                       side = "SELL",
                                       type = "LIMIT",
                                        price = "300.00",
                                       quantity = "0.05",
                                       timeInForce = "GTC")**

**order = client.create_order(symbol = symbol = "BNBUSDT",
                                        side = "BUY",
                                        type = "STOP_LOSS_LIMIT",
                                        stopPrice = "200.00",
                                        price = "180.00", quantity = "0.05",
                                        timeInForce = "GTC")**

<br>

Parameters
<li><b>symbol</b> is the name of the pair we want to trade.
<li><b>side</b> shows either we want to "BUY" or "SELL".
<li><b>type</b> is the type of the transaction. This can be "MARKET" to trade at the current price, "LIMIT" to trade, "STOP_LOSS_LIMIT" to create an order that is activated a certain price.

<br><br>

**Bot Structure**

Bot structure may be the most important part of creating a bot. Before coding this should be designed carefully. The basic steps are as follows:

<li> Defining a variable that shows our current situation, i.e., are we ready to buy or sell. This is crucial because we do not want to buy or sell more than once, when our price criterion is met.
<li> Creating a loop for constantly importing the most recent data.
<li> Checking current order situation, i.e., is an open order has been filled.
<li> Analyzing the most recent data to decide our price that we are willing to trade.
<li> If there is no open order, then create an order using "LIMIT" or continue to create an order later when the price criterion is met.
<li> Change our current variable accordingly.
<br><br>

<img src='bot-flowchart.png'/>

<br><br>

**Trading Strategy**

Before making a trading bot, we should decide our strategy, at which price we would buy or sell. There are countless possibilities to decide that. This work focuses on trading bot, thus, we create a simple grid function.

In general, technical indicators are used to determine the target price. Some technical indicators can be found another work in this repository. The link will be provided as soon as it is uploaded.

In [90]:
def price_finder_grid(side, buy_price=224.3, sell_price=224.5):
    
    if side == "BUY":
        return buy_price
    if side == "SELL":
        return sell_price

The price_finder function is basically a implemantation of grid trade that is buying/selling at given prices.

<br><br> Following this structure, we can make a trader bot. The explanations will be given in the function.

**Important Note:** The bot will not stop unless you force it, for example, by interrupting kernel.

In [93]:
def trader(currency, amount=0, side="BUY", commission=0.00075):
    
    #define current position
    open_order = False
    
    #checking the parameters are given correctly
    side = side.upper()
    if side not in ["BUY", "SELL"]:
        print("Please enter a valid 'side' Value")
        return
    if side == "SELL":    #This is placed for avoiding error when calculating Profit
        buyprice=0
    
    amount = float(amount)
    commission = float(commission)
    
    print("\nBOT started!\n")
    
    
    
    
    #Importing recent data
    df = klines_table(currency, "15m", 15*15)     #15m interval data for last 15*15 minutes
    last_time = df.index[-1]                      #Last recorded time interval
    last_price = df["Close"][-1]                  #Most recent price
    
    
    
    
    #Loop begins here
    while True:      
        
        #importing recent data
        df = klines_table(currency, "15m", 15*15)     #15m interval data for last 15*15 minutes
            
            
            
        
        # If there is an open order, change current side if filled, cancel it otherwise to be updated later
        if open_order == True:
            open_order = False
            currentOrder = client.get_order(symbol=currency,orderId=orderId) # Existing Order Info
            
            if currentOrder['status']=='FILLED':  #Check if the existing order is filled           

                #Change the recent side
                if side == "BUY":
                    side = "SELL"
                    buyprice = float(client.get_my_trades(symbol = currency)[-1]["price"])   #Get the price where the order is filled at
                    print("\n\nBuy order completed at $ {:.4f}\n\n".format(buyprice))     
                    
                elif side == "SELL":
                    side = "BUY"
                    sellprice = float(client.get_my_trades(symbol = currency)[-1]["price"])   #Get the price where the order is filled at
                    print("\n\nSell order completed: $ {:.4f}\n\n".format(sellprice))
                    print("***** PROFIT: $ {:.4f}".format(amount*((sellprice-buyprice)-(sellprice+buyprice)*commission)))
                    print ("% {:.4f}\n\n".format(((sellprice-buyprice)-(sellprice+buyprice)*commission)/buyprice*100) )
            
            #If current order is not filled then cancel the order to be updated later
            else:
                result = client.cancel_order(symbol=currency,
                                                 orderId=orderId)
                print("{} order is canceled.\n".format(side))

        
        
        
        # Setting the target price we want to buy or sell
        target_price = price_finder_grid(side=side)
        
        
        
        
        # Create a buy/sell order
        order = client.create_order(symbol = currency,
                                                side = side,
                                                type = "LIMIT",
                                                price = "{:.3f}".format(target_price),
                                                quantity = str(amount),
                                                timeInForce = "GTC")
        
        print(datetime.fromtimestamp(time.time()).strftime("%H:%M:%S"))
        print("{0} order placed for {1} {2} at $ {3} // Current Price: {4}".format(side, amount, currency, target_price, df["Close"][-1]))
        orderId = order["orderId"]
        print("OrderId: ",orderId)
        currentOrder = client.get_order(symbol=currency,orderId=orderId)
        open_order = True
        
        
        
        #Wait between each loop. 
        #Too many requests from BINANCE API may result in banning or cutting connection.
        time.sleep(15)

                    
    
        

<br><br> Let's check the function in real life.

In [91]:
trader("BNBUSDT", amount = "0.05")


BOT started!

21:20:22
BUY order placed for 0.05 BNBUSDT at $ 224.3 // Current Price: 224.2
OrderId:  4184710000


Buy order completed at $ 224.2000


21:20:39
SELL order placed for 0.05 BNBUSDT at $ 224.5 // Current Price: 224.2
OrderId:  4184710332
SELL order is canceled.

21:20:58
SELL order placed for 0.05 BNBUSDT at $ 224.5 // Current Price: 224.1
OrderId:  4184710597
SELL order is canceled.

21:21:17
SELL order placed for 0.05 BNBUSDT at $ 224.5 // Current Price: 224.2
OrderId:  4184710916
SELL order is canceled.

21:21:35
SELL order placed for 0.05 BNBUSDT at $ 224.5 // Current Price: 224.2
OrderId:  4184711310
SELL order is canceled.

21:21:54
SELL order placed for 0.05 BNBUSDT at $ 224.5 // Current Price: 224.2
OrderId:  4184711532
SELL order is canceled.

21:22:14
SELL order placed for 0.05 BNBUSDT at $ 224.5 // Current Price: 224.1
OrderId:  4184712333
SELL order is canceled.

21:22:32
SELL order placed for 0.05 BNBUSDT at $ 224.5 // Current Price: 224.1
OrderId:  418471322

KeyboardInterrupt: 

<br><br> The bot is working as expected. I have not implemented an advanced strategy and keep the difference between buy and sell prices too narrow to test all code lines. I made a small loss because the commission was greater than my gross profit. 

<br><br><br>

## 3. CONCLUSION AND DISCUSSION

Trader bots allow users to buy or sell stocks automatically at a given price or a dynamic price. We can implement a strategy to trade with bots. This can be finding a price with trading indicators, giving a selling order after buying order is filled or anything you can think of. My motivation was to eliminate human emotions while trading. 

In this work, I have created a trader bot for binance that makes an actual trade in real life. I explained all steps and created a flowchart to make the process clear. This bot can be improved far more with your imaginations and skills.

**See also** Technical Indicators for Financial Analysis with Python (Jupyter): (the link will be uploaded.)

**See also** Gate.io Trader Bot with Python: (the link will be uploaded.)

**See also** Data Visualization for Financial Analysis with Python (Jupyter): (the link will be uploaded.)